<a href="https://colab.research.google.com/github/jaswanth99999/GPT/blob/master/GPT_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.5 MB/s eta 0:00:00


In [2]:
!python --version

Python 3.8.10


In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import torch
from torch.utils.data import Dataset 
import random
import time
import datetime
import random
from transformers import GPT2LMHeadModel, GPT2Config
import numpy as np
from torch.utils.data import random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [4]:
req='/content/data.txt'
with open('/content/np.txt') as f:
    req = f.read()


In [5]:
all_sentences = req.split(".")


In [6]:
# import re
# # "semicolon or (a comma followed by a space)"
# pattern = re.compile(r",")

# # "(semicolon or a comma) followed by a space"
# # pattern = re.compile(r"[;,] ")

# all_sentences=pattern.split(req)

In [7]:
len(all_sentences)

16600

In [8]:
all_sentences = [s.replace("\r","") for s in all_sentences]

In [9]:
print("sample size : ",len(all_sentences))
print("samples     : " )
all_sentences[0:10]

sample size :  16600
samples     : 


['The 2020 Summer Olympics (Japanese: 2020年夏季オリンピック, Hepburn: Nisen Nijū-nen Kaki Orinpikku), officially the Games of the XXXII Olympiad (第三十二回オリンピック競技大会, Dai Sanjūni-kai Orinpikku Kyōgi Taikai) and also known as Tokyo 2020 (東京2020, Tōkyō Nii Zero Nii Zero), was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021',
 ' Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013',
 'The Games were originally scheduled to take place from 24 July to 9 August 2020, but due to the global COVID-19 pandemic, on 24 March 2020 the event was postponed to 2021, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled)',
 ' However, the event retained the Tokyo 2020 branding for marketing purposes',
 ' It was largely held behind closed doors with no public spectators permitted due to the declaration 

In [10]:
from transformers import GPT2Tokenizer
#get pretrained tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<sos>', eos_token='<eos>', pad_token='<pad>')

#tokenizer some samples
print( tokenizer.encode("Japan Tokyo") )
print( tokenizer.encode("Japan") )
print( tokenizer.encode("japan tokyo") )
print( tokenizer.encode("japan") )
print( tokenizer.encode("tokyo") )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[16504, 11790]
[16504]
[73, 2674, 284, 2584, 78]
[73, 2674]
[83, 482, 8226]


In [11]:
max_len = max([len(tokenizer.encode(s)) for s in all_sentences])

print(f"max_len {max_len}")

max_len 1022


In [12]:
def tokenize_seq(sent,tokenizer,max_length):
  return tokenizer('<sos>'+ sent + '<eos>', truncation=True, max_length=max_length, padding="max_length")

class JapanDataset(Dataset):

  def __init__(self, sentences, tokenizer, gpt2_type="gpt2", max_length=max_len):

    self.tokenizer = tokenizer 
    self.input_ids = []
    self.attn_masks = []

    for sentence in sentences:      
      encodings = tokenize_seq(sentence,tokenizer,max_length)
            
      self.input_ids.append(torch.tensor(encodings['input_ids']))
      self.attn_masks.append(torch.tensor(encodings['attention_mask']))
    
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx]   

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [13]:
import gc
gc.collect()

4

In [14]:
#create an instance of Dataset
dataset = JapanDataset(all_sentences, tokenizer, max_length=max_len)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_set, val_set = random_split(dataset, [train_size, val_size])
print("train_size :",train_size)
print("val_size   :",val_size)

gc.collect() 

train_size : 14940
val_size   : 1660


0

In [15]:
dataset[0]

(tensor([50257,   464, 12131,  ..., 50259, 50259, 50259]),
 tensor([1, 1, 1,  ..., 0, 0, 0]))

In [16]:
train_dataloader = DataLoader(train_set,  sampler = RandomSampler(train_set), batch_size = 1)
validation_dataloader = DataLoader(val_set, sampler = SequentialSampler(val_set), batch_size = 1 )

In [17]:
# Create default config
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)
model.resize_token_embeddings(len(tokenizer))

device = torch.device("cuda")
model.cuda()


optimizer = torch.optim.Adam(model.parameters(),lr = 0.0005)
model = model.to(device)

In [18]:
#at every step i want to check if generations are getting better.
def eval_keywords(keywords):
  model.eval()
  for keyword in keywords:
    input_seq = "<sos> " + keyword
    generated = torch.tensor(tokenizer.encode(input_seq)).unsqueeze(0)
    generated = generated.to(device)
    sample_outputs = model.generate(
                                generated, 
                                do_sample=True,   
                                top_k=30, 
                                max_length = 50,
                                top_p=0.90, 
                                num_return_sequences=2
                                )
    for i, sample_output in enumerate(sample_outputs):
      print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

keywords = ["Osaka","Japan","Kyoto","Yokohama","Kanto","Nikko","Japan has","Tokyo is the","Osaka is the","Kyoto is the"]

In [19]:
#call model with a batch of input
def process_one_batch(batch):
  b_input_ids = batch[0].to(device)
  b_labels = batch[0].to(device)
  b_masks = batch[1].to(device)
  outputs  = model(b_input_ids,  attention_mask = b_masks,labels=b_labels)
  return outputs

#do one epoch for training
def train_epoch():
  t0 = time.time()
  total_train_loss = 0
  model.train()
  for step, batch in enumerate(train_dataloader):
        
        model.zero_grad()        
        outputs = process_one_batch( batch)
        loss = outputs[0]  
        batch_loss = loss.item()
        total_train_loss += batch_loss

        loss.backward()
        optimizer.step()

        
  avg_train_loss = total_train_loss / len(train_dataloader)  
  print("avg_train_loss",avg_train_loss)  
  elapsed_time = format_time(time.time() - t0)
  print("elapsed time for 1 training epoch : ",elapsed_time)

#do one epoch for eval
def eval_epoch():
  t0 = time.time()
  total_eval_loss = 0
  nb_eval_steps = 0
  # Evaluate data for one epoch
  for batch in validation_dataloader:            
        
    with torch.no_grad():        
      outputs = process_one_batch( batch)
      loss = outputs[0]              
      batch_loss = loss.item()
      total_eval_loss += batch_loss         

  avg_val_loss = total_eval_loss / len(validation_dataloader)
  print("avg_val_loss",avg_val_loss) 
  elapsed_time = format_time(time.time() - t0)
  print("elapsed time for 1 eval epoch : ",elapsed_time)

In [20]:
train_epoch()
eval_epoch()
eval_keywords( keywords )

avg_train_loss 0.12699439380268993
elapsed time for 1 training epoch :  1:33:05


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


avg_val_loss 0.10403368329118758
elapsed time for 1 eval epoch :  0:03:15


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0:  Osaka won the last round of five Olympic medals (two gold medals at the 2020 Games) and was won by defeating him and won two medals, respectively
1:  Osaka, who competed on the fourth round, with a maximum score of 5, but had been eliminated in the quarterfinals


...

0:  Japan has been held at every edition since 1952 A major result of the 2004 Games 2020 Games
1:  Japan has been held in all matches, with a double victory game since 1908 as well as a win by Taiwan (when he was


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0:  Tokyo is the most common Japanese official appearance and has appeared at every edition since the 1980 Summer Olympics in history, with
1:  Tokyo is the first time that a Japanese swimmer who qualifies to the round (or of 32) was named on June 6 April 2021 2021


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0:  Osaka is the reigning World Champion, with an upset victory in this event he won the first gold medal, most in the event since history and the debut in the
1:  Osaka is the reigning world record holder that was won the game, after the Olympics is held, to win
0:  Kyoto is the first gold medal and has been held that this freestyle at the event was won by the Danish athlete in this
1:  Kyoto is the world's twenty-sixth consecutive appearance, having first competed since 1984 has been won in a previous edition of
